In [1]:
import numpy as np
import sympy as sp

In [32]:
# metros
longitud_cables = 1.5
# pulgadas
diametro_cables = 1 / 4
# mm 2
area_cables = np.pi * (diametro_cables * 25.4 / 1000) ** 2 / 4
# pulgadas
diametro_pasadores = 1 / 2
# mm 2
area_pasadores = np.pi * (diametro_pasadores * 25.4 / 1000) ** 2 / 4

codigo = "2200269"
X = sum([int(i) for i in codigo])
# metros
L = 0.2 * X

FS_fluencia = float(f"1.{X}")
FS_resistencia_ultima = float(f"1.{X + 3}")

angulo_BC = 40 * np.pi / 180
angulo_BE = 50 * np.pi / 180

In [33]:
# propiedades material
# MPa
esfuerzo_ultimo = 400
esfuerzo_fluencia_traccion = 250
esfuerzo_fluencia_corte = 145
# GPa
E = 200

In [34]:
# Equilibrio
Ax, Ay, BC, BD, BE, W = sp.symbols('Ax Ay BC BD BE W')

Fx = Ax - BC * sp.sin(angulo_BC) + BE * sp.sin(angulo_BE)
equilibrio_X = sp.Eq(Fx, 0)
equilibrio_X

Eq(Ax - 0.642787609686539*BC + 0.766044443118978*BE, 0)

In [35]:
Fy = Ay - W * L + BC * sp.cos(angulo_BC) + BE * sp.cos(angulo_BE) + BD
equilibrio_Y = sp.Eq(Fy, 0)
equilibrio_Y

Eq(Ay + 0.766044443118978*BC + BD + 0.642787609686539*BE - 4.2*W, 0)

In [36]:
M_a = - W * L ** 2 / 2 + BC * sp.cos(angulo_BC) * L + BE * sp.cos(angulo_BE) * L + BD * L
equilibrio_momentos = sp.Eq(M_a, 0)
equilibrio_momentos

Eq(3.21738666109971*BC + 4.2*BD + 2.69970796068347*BE - 8.82*W, 0)

In [37]:
# por compatibilidad de desplazamientos
equivalencia_BC = sp.Eq(BC - BD * sp.cos(angulo_BC), 0)
expresion_equivalencia_BC = BD * sp.cos(angulo_BC)
equivalencia_BC

Eq(BC - 0.766044443118978*BD, 0)

In [38]:
equivalencia_BE = sp.Eq(BE - BD * sp.cos(angulo_BE), 0)
expresion_equivalencia_BE = BD * sp.cos(angulo_BE)
equivalencia_BE

Eq(-0.642787609686539*BD + BE, 0)

In [39]:
# obtener BD del equilibrio de momentos
ecuacion = sp.Eq(M_a.subs([(BC, expresion_equivalencia_BC), (BE, expresion_equivalencia_BE)]), 0)
ecuacion

Eq(8.4*BD - 8.82*W, 0)

In [40]:
BD_vs_W = sp.solve(ecuacion)[0][BD]
BD_vs_W

1.05*W

In [41]:
F_BC = expresion_equivalencia_BC.subs(BD, BD_vs_W)
F_BE = expresion_equivalencia_BE.subs(BD, BD_vs_W)

fuerzas_cables = [F_BC, BD_vs_W, F_BE]
fuerzas_cables

[0.804346665274927*W, 1.05*W, 0.674926990170866*W]

In [48]:
w_max = []
Fxx = sp.symbols('Fxx')
for fuerza in fuerzas_cables:
    # analisis de esfuerzo normal en cables
    ecuacion_esfuerzo_normal = sp.Eq(Fxx / area_cables, esfuerzo_ultimo * 10 ** 6 / FS_resistencia_ultima)
    w_cables = sp.solve(ecuacion_esfuerzo_normal)[0] / 1000
    # cortante simple en pasadores superiores
    ecuacion_esfuerzo_cortante = sp.Eq(Fxx / area_pasadores, esfuerzo_fluencia_corte * 10 ** 6 / FS_fluencia)
    w_pasadores = sp.solve(ecuacion_esfuerzo_cortante)[0] / 1000
    print(w_cables, w_pasadores)
    w_max.append(w_cables)
    w_max.append(w_pasadores)

print("Carga distribuida máxima en los cables [kN / m]")
min(w_max)
print()


10.2158765947076 15.1802860473424
10.2158765947076 15.1802860473424
10.2158765947076 15.1802860473424
Carga distribuida máxima en los cables [kN / m]



In [43]:
# cortante doble en los apoyos de la barra
Ay_eq = Fy.subs([(BC, expresion_equivalencia_BC), (BE, expresion_equivalencia_BE), (BD, BD_vs_W)])
Ay_vs_W = sp.solve(sp.Eq(Ay_eq, 0))
Ay_vs_W = Ay_vs_W[0][Ay]
Ay_vs_W

2.1*W

In [44]:
Ax_eq = Fx.subs([(BC, expresion_equivalencia_BC), (BE, expresion_equivalencia_BE), (BD, BD_vs_W)])
Ax_vs_W = sp.solve(sp.Eq(Ax_eq, 0))
Ax_vs_W = Ax_vs_W[0][Ax]
Ax_vs_W

-1.16573417585641e-16*W

In [45]:
# fuerza en el apoyo A
F_apoyo_A = sp.sqrt(Ay_vs_W ** 2 + Ax_vs_W ** 2)
F_apoyo_A

2.1*sqrt(W**2)

In [46]:
# fuerza en el punto de conexión de los cables
F_x_cortante_doble = expresion_equivalencia_BE.subs(BD, BD_vs_W) * sp.sin(angulo_BE) - expresion_equivalencia_BC.subs(BD, BD_vs_W) * sp.sin(angulo_BC)
F_y_cortante_doble = expresion_equivalencia_BE.subs(BD, BD_vs_W) * sp.cos(angulo_BE) + expresion_equivalencia_BC.subs(BD, BD_vs_W) * sp.cos(angulo_BC) + BD_vs_W
F_apoyo_B = sp.sqrt(F_x_cortante_doble ** 2 + F_y_cortante_doble ** 2)
F_apoyo_B

2.1*sqrt(W**2)

In [47]:
fuerzas_apoyos = [F_apoyo_A, F_apoyo_B]
w_max_apoyos = []
for fuerza in fuerzas_apoyos:
    ecuacion_esfuerzo_cortante = sp.Eq(fuerza / area_pasadores, esfuerzo_fluencia_corte * 10 ** 6 / FS_fluencia)
    w_pasadores = sp.solve(ecuacion_esfuerzo_cortante)[0] / 1000
    w_max_apoyos.append(w_pasadores)

print("Carga distribuida máxima por pasadores [kN / m]")
carga_maxima = min(w_max)
carga_maxima

Carga distribuida máxima por pasadores [kN / m]


9.72940628067390

# Cálculo de fuerzas y deformaciones

In [18]:
valor_BE = expresion_equivalencia_BE.subs([(BD, BD_vs_W), (W, carga_maxima)])
deformacion_BE = (valor_BE * 1000 * longitud_cables) / (area_cables * E * 10 ** 9)
# fuerza en kN y deformacion en mm
valor_BE, deformacion_BE * 1000

(6.12227987404836, 1.44989686395460)

In [19]:
valor_BC = expresion_equivalencia_BC.subs([(BD, BD_vs_W), (W, carga_maxima)])
deformacion_BC = (valor_BC * 1000 * longitud_cables) / (area_cables * E * 10 ** 9)
valor_BC, deformacion_BC * 1000

(7.29624903476436, 1.72791979650897)

In [20]:
valor_BD = BD_vs_W.subs(W, carga_maxima)
deformacion_BD = (valor_BD * 1000 * longitud_cables) / (area_cables * E * 10 ** 9)
valor_BD, deformacion_BD * 1000

(9.52457667476500, 2.25563909774436)

In [21]:
valor_Ax = Fx.subs([(BE, valor_BE), (BC, valor_BC)])
valor_Ax = sp.solve(sp.Eq(valor_Ax, 0))[0]
valor_Ax

-8.88178419700125e-16

In [24]:
valor_Ay = Fy.subs([(BE, valor_BE), (BC, valor_BC), (BD, valor_BD), (W, carga_maxima)])
valor_Ay = sp.solve(sp.Eq(valor_Ay, 0))[0]
valor_Ay

19.0491533495300